In [2]:
!nvidia-smi

Tue Mar 11 10:39:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.04             Driver Version: 570.124.04     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 4000 Ada Gene...    Off |   00000000:01:00.0 Off |                  Off |
| 30%   40C    P8             17W /  130W |     149MiB /  20475MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%cd ..

from mingpt.bpe import BPETokenizer
from mingpt.model import GPT

import torch
import pandas as pd

from datasets import load_dataset
from torch.utils.data import DataLoader

/users/eleves-a/2022/brahim.touayouch/projects/MiniGPT/.venv/lib64/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/users/eleves-a/2022/brahim.touayouch/projects/MiniGPT


/users/eleves-a/2022/brahim.touayouch/projects/MiniGPT/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class StoryDataset:
    def __init__(self, data, tokenizer, block_size=128):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.data1 = [
            self.format_example(data_point["text"]) for data_point in data
        ]

    def format_example(self, text):
        tokens = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=self.block_size)
        tokens = tokens.squeeze(0)
        
        return tokens

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, idx):
        """
        Returns:
            x (torch.Tensor): Input tokens (question + answer prompt).
            y (torch.Tensor): Target tokens (shifted output).
        """
        if isinstance(idx, int):
            tokens = self.data1[idx]
            x = tokens[:-1].clone().detach()
            y = tokens[1:].clone().detach()
            return x, y
        
        elif isinstance(idx, list) or isinstance(idx, torch.Tensor):
            batch_tokens = [self.data1[i] for i in idx]
            x_batch = [tokens[:-1].clone().detach() for tokens in batch_tokens]
            y_batch = [tokens[1:].clone().detach() for tokens in batch_tokens]
            return x_batch, y_batch
        
        else:
            raise TypeError(f"Invalid index type: {type(idx)}")


In [5]:
data = load_dataset('roneneldan/TinyStories')
data_frame = pd.DataFrame(data["train"])

In [6]:
data_frame

,text
0,"One day, a little girl named Lily found a need..."
1,"Once upon a time, there was a little car named..."
2,"One day, a little fish named Fin was swimming ..."
3,"Once upon a time, in a land full of trees, the..."
4,"Once upon a time, there was a little girl name..."
...,...
2119714,"Once upon a time, in a small town, there lived..."
2119715,"Once upon a time, there was a little boy named..."
2119716,"Once upon a time, there was a big tree. Under ..."
2119717,"Once upon a time, there was a little girl name..."


In [7]:
data_frame.iloc[0]['text']

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'

In [8]:
train_testvalid = data["train"].train_test_split(test_size=0.9, seed=42)

test_valid = train_testvalid["test"].train_test_split(test_size=0.5, seed=42)

train_data = train_testvalid["train"]
val_data = test_valid["train"]
test_data = test_valid["test"]

print(f"Train size: {len(train_data)}, Validation size: {len(val_data)}, Test size: {len(test_data)}")

Train size: 211971, Validation size: 953874, Test size: 953874


In [9]:
tokenizer = BPETokenizer()

train_dataset = StoryDataset(train_data.select(range(8000)), tokenizer=tokenizer, block_size=256)
test_dataset = StoryDataset(test_data.select(range(4000)), tokenizer=tokenizer, block_size=256)
val_dataset = StoryDataset(val_data.select(range(3000)), tokenizer=tokenizer, block_size=256)

# Load the model

In [10]:
model_type = 'gpt2'
device = 'cuda'

model = GPT.from_pretrained(model_type)
model.to(device)

number of parameters: 124.44M


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(

# Evaluations

In [11]:
def generate(prompt='', num_samples=1, steps=20, do_sample=True):
    tokenizer = BPETokenizer()

    if prompt == '':
        x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
    else:
        x = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=len(prompt.split())).to(device)

    x = x.expand(num_samples, -1)

    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)

    endoftext_token_id = tokenizer.encoder.encoder['<|endoftext|>']

    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())

        out_tokens = y[i].cpu().squeeze().tolist()
        
        if endoftext_token_id in out_tokens:
            end_pos = out_tokens.index(endoftext_token_id)
            out = tokenizer.decode(y[i].cpu().squeeze()[:end_pos])

        print('\n' + '-' * 80)
        print(out)

### Generation Evaluation

In [17]:
generate(prompt='One day, a little girl named Lily found a', num_samples=1, steps=100)


--------------------------------------------------------------------------------
One day, a little girl named Lily found her feet in a pool with no money.

"It was because I said things like, 'Don't tell this to them or anyone, I'm sure you would get in trouble," he remembers.

"It happened last week. I knew it was happening to this girl and her story was good. It was a few days ago but it was very good."

Lily was taken seriously after her story hit the news.

"That girl took care of herself as


In [20]:
generate(prompt='One day,', num_samples=1, steps=100)


--------------------------------------------------------------------------------
One day. The week before.

That's it.

A day

When I'm out

That's it.


I want to make a video

A moment.

A moment

A moment

How I like

What I like.

How I like.

How much I like.

I love

Who's the best.

What I like?

Who's that.

What I


In [21]:
generate(prompt='Once upon a time', num_samples=1, steps=100)


--------------------------------------------------------------------------------
Once upon a time, it is not a bad idea to send to the first woman to her new. I have sent two boys to the first time but was told of him last time the first time. I think it's best I do things to be as honest with him and be as honest as to what will happen to him. I feel like he is having a bad time but that is the time that he should be. My boys are trying so I believe that. I feel we are trying. The time is


### Test Loss and Perplexity Evaluation

In [22]:
@torch.no_grad()
def evaluate(model, dataset, partition, batch_size=24, device='cuda'):
    model.eval()
    test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    total_loss = 0.0
    num_batches = 0

    for batch in test_loader:
        batch = [t.to(device) for t in batch]
        x, y = batch
        _, loss = model(x, y)

        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    perplexity = torch.exp(torch.tensor(avg_loss))
    print(f"{partition} Loss: {avg_loss:.4f}")
    print(f"{partition} Perplexity: {perplexity:.4f}")
    return avg_loss, perplexity

In [23]:
train_loss, perplexity = evaluate(model=model, dataset=train_dataset, partition='Train')

Train Loss: 5.1534
Train Perplexity: 173.0197


In [24]:
val_loss, perplexity = evaluate(model=model, dataset=val_dataset, partition='Validation')

Validation Loss: 5.0995
Validation Perplexity: 163.9380


In [25]:
test_loss, perplexity = evaluate(model=model, dataset=test_dataset, partition='Test')

Test Loss: 5.1494
Test Perplexity: 172.3307
